## Setup: Finnhub API Key

This notebook uses real market data from [Finnhub](https://finnhub.io/) (free tier: 60 API calls/min).

**To use real data:**
1. Get a free API key at https://finnhub.io/register
2. Set environment variable: `export FINNHUB_API_KEY=your_key_here`
3. Or enter the key when prompted

**Without a key:** The notebook falls back to synthetic data generation.


# Triangular Arbitrage — Example

Notebook example ready to push. It attempts to use the Rust connector if available (via python.rust_bridge), otherwise falls back to pure-Python implementations. Includes equations, strategy, backtest (PnL, Sharpe, Drawdown) and visualisations.

In [ ]:
# Imports + safe fallbacks
import json
import math
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

RUST_AVAILABLE = False
try:
    from python.rust_bridge import parse_orderbook as rust_parse_orderbook, compute_triangular_opportunity as rust_tri
    RUST_AVAILABLE = True
except Exception:
    # rust not available; fall back to python implementations below
    RUST_AVAILABLE = False

try:
    from python.backtest.core import Backtest, sharpe_ratio, max_drawdown
except Exception:
    # Minimal local Backtest fallback (keeps notebook runnable)
    class Backtest:
        def __init__(self, initial_cash=100000.0):
            self.cash = initial_cash
            self.positions = {}
            self.trades = []
            self.history = []
        def execute_trade(self, timestamp, symbol, qty, price, side):
            cost = qty * price
            if side.lower() == 'buy':
                self.cash -= cost
                prev = self.positions.get(symbol, (0.0, 0.0))
                new_qty = prev[0] + qty
                new_avg = (prev[0]*prev[1] + qty*price) / (new_qty if new_qty!=0 else 1)
                self.positions[symbol] = (new_qty, new_avg)
            else:
                self.cash += cost
                prev = self.positions.get(symbol, (0.0, 0.0))
                new_qty = prev[0] - qty
                if new_qty <= 0:
                    self.positions.pop(symbol, None)
                else:
                    self.positions[symbol] = (new_qty, prev[1])
            self.trades.append({"ts": timestamp, "symbol": symbol, "qty": qty, "price": price, "side": side})
            self.mark_to_market(timestamp, {})
        def mark_to_market(self, timestamp, market_prices: dict):
            equity = self.cash
            for s,(qty,avg) in self.positions.items():
                price = market_prices.get(s, avg)
                equity += qty*price
            self.history.append((timestamp, equity))
            return equity
        def results_df(self):
            df = pd.DataFrame(self.history, columns=["ts", "equity"]).set_index("ts")
            df["returns"] = df["equity"].pct_change().fillna(0.0)
            df["cum_return"] = (1 + df["returns"]).cumprod() - 1
            return df

    def sharpe_ratio(returns, freq=252):
        if len(returns) < 2:
            return 0.0
        mu = returns.mean() * freq
        sigma = returns.std() * math.sqrt(freq)
        return mu / sigma if sigma != 0 else 0.0

    def max_drawdown(equity_curve):
        roll_max = equity_curve.cummax()
        drawdown = (equity_curve - roll_max) / roll_max
        return drawdown.min()

# Import Finnhub helper
try:
    from python.finnhub_helper import fetch_historical_simulation, create_orderbook_from_quote, get_finnhub_api_key
    FINNHUB_HELPER_AVAILABLE = True
except Exception as e:
    print(f"Warning: Could not import Finnhub helper: {e}")
    FINNHUB_HELPER_AVAILABLE = False

print('Rust available:', RUST_AVAILABLE)
print('Finnhub helper available:', FINNHUB_HELPER_AVAILABLE)


## Triangular arbitrage math (intuitive)

Three markets: A/B, B/C, C/A. If you start with 1 unit of A and after sequentially trading A->B, B->C, C->A you end with >1 unit, there is arbitrage.
Profit ≈ (price_AB_sell^{-1} * price_BC_sell^{-1} * price_CA_sell^{-1}) - 1 when chaining rates. In practice use best bid/ask and account for fees and slippage.


In [ ]:
# Synthetic data generation: produce snapshots that occasionally contain arbitrage
def make_ob(mid, spread=0.001, size=100):
    # simple orderbook top-of-book
    bid = round(mid * (1 - spread/2), 8)
    ask = round(mid * (1 + spread/2), 8)
    return {"bids": [[bid, size]], "asks": [[ask, size]]}

snapshots = []
random.seed(42)
for t in range(200):
    # base mid prices
    m_ab = 1.0 + random.normalvariate(0, 0.0005)
    m_bc = 2.0 + random.normalvariate(0, 0.0005)
    # occasionally inject a small arbitrage by tweaking one mid
    if random.random() < 0.06:
        m_ca = 1.0 / (m_ab * m_bc) * 1.0015  # create positive loop
    else:
        m_ca = 1.0 / (m_ab * m_bc) * (1 + random.normalvariate(0, 0.0002))
    ob1 = make_ob(m_ab)
    ob2 = make_ob(m_bc)
    ob3 = make_ob(m_ca)
    snapshots.append((json.dumps(ob1), json.dumps(ob2), json.dumps(ob3)))

len(snapshots)


In [ ]:
# Generate triangular market data: A-B, B-C, C-A using real Finnhub data
# We'll use BTC, ETH, BNB as the three assets for triangular arbitrage
# Triangular path: BTC->ETH, ETH->BNB, BNB->BTC

# Get Finnhub API key (from env or user input)
api_key = get_finnhub_api_key() if FINNHUB_HELPER_AVAILABLE else None

if FINNHUB_HELPER_AVAILABLE and api_key:
    print("Fetching real market data from Finnhub (this may take a moment)...")
    # Fetch historical simulation data for 3 pairs
    symbols = ['BINANCE:BTCUSDT', 'BINANCE:ETHUSDT', 'BINANCE:BNBUSDT']
    df = fetch_historical_simulation(symbols, periods=200, api_key=api_key)
    
    # Create orderbook snapshots from price data
    # For triangular arbitrage: calculate cross rates
    # AB = BTC/ETH, BC = ETH/BNB, CA = BNB/BTC
    snapshots = []
    
    for idx, row in df.iterrows():
        btc_price = row['BINANCE:BTCUSDT_mid']
        eth_price = row['BINANCE:ETHUSDT_mid']
        bnb_price = row['BINANCE:BNBUSDT_mid']
        
        # Calculate cross rates
        m_ab = btc_price / eth_price  # BTC per ETH
        m_bc = eth_price / bnb_price  # ETH per BNB
        m_ca = bnb_price / btc_price  # BNB per BTC
        
        # Create orderbooks with small spreads
        spread_pct = 0.0005  # 5 bps
        
        ob1 = create_orderbook_from_quote(
            bid=m_ab * (1 - spread_pct),
            ask=m_ab * (1 + spread_pct),
            size=10.0
        )
        ob2 = create_orderbook_from_quote(
            bid=m_bc * (1 - spread_pct),
            ask=m_bc * (1 + spread_pct),
            size=10.0
        )
        ob3 = create_orderbook_from_quote(
            bid=m_ca * (1 - spread_pct),
            ask=m_ca * (1 + spread_pct),
            size=10.0
        )
        
        snapshots.append((json.dumps(ob1), json.dumps(ob2), json.dumps(ob3)))
    
    print(f"Generated {len(snapshots)} snapshots from Finnhub data")
    print(f"Price range - BTC: ${df['BINANCE:BTCUSDT_mid'].min():.0f}-${df['BINANCE:BTCUSDT_mid'].max():.0f}")
    print(f"Price range - ETH: ${df['BINANCE:ETHUSDT_mid'].min():.0f}-${df['BINANCE:ETHUSDT_mid'].max():.0f}")
    print(f"Price range - BNB: ${df['BINANCE:BNBUSDT_mid'].min():.0f}-${df['BINANCE:BNBUSDT_mid'].max():.0f}")
    
else:
    print("Finnhub not available - using synthetic data fallback")
    # Original synthetic data generation
    def make_ob(mid, spread=0.001, size=100):
        bid = mid - spread * mid
        ask = mid + spread * mid
        return {"bids": [[bid, size]], "asks": [[ask, size]]}

    import random
    random.seed(42)
    snapshots = []
    for i in range(200):
        # mid prices for A-B, B-C, C-A
        m_ab = 1.0 + random.gauss(0, 0.001)
        m_bc = 2.0 + random.gauss(0, 0.002)
        # Occasionally inject an arbitrage opportunity
        if random.random() < 0.06:
            m_ca = 0.49 + random.gauss(0, 0.001)
        else:
            implied = 1.0 / (m_ab * m_bc)
            m_ca = implied + random.gauss(0, 0.0005)
        
        ob1 = make_ob(m_ab)
        ob2 = make_ob(m_bc)
        ob3 = make_ob(m_ca)
        snapshots.append((json.dumps(ob1), json.dumps(ob2), json.dumps(ob3)))
    
    print(f"Generated {len(snapshots)} synthetic snapshots")


In [ ]:
# Backtest: detect opportunities and simulate trades
bt = Backtest(initial_cash=100000)
market_prices = {}
threshold = 0.0008  # require at least 0.08% loop profit
trade_size_A = 10.0  # units of A
ts = 0
for ob1_json, ob2_json, ob3_json in snapshots:
    profit, route = triangular_strategy(ob1_json, ob2_json, ob3_json)
    # record a mark-to-market using mid of A from ob1
    ob1 = json.loads(ob1_json)
    mid_A = (ob1['bids'][0][0] + ob1['asks'][0][0]) / 2
    market_prices['A'] = mid_A
    bt.mark_to_market(ts, { 'A': mid_A })
    if profit > threshold:
        # simulate executing the triangular cycle: A->B, B->C, C->A
        # simplified: sell trade_size_A at bid price
        bid_A = ob1['bids'][0][0]
        bt.execute_trade(ts, 'A', trade_size_A, bid_A, 'sell')
        # buy back at ask price after loop to close
        ask_A = ob1['asks'][0][0]
        # simulate that we re-acquire A at a better effective price proportional to profit
        effective_buy_price = ask_A / (1 + profit)
        bt.execute_trade(ts, 'A', trade_size_A, effective_buy_price, 'buy')
    ts += 1

df = bt.results_df()
print('Trades:', len(bt.trades))
print('Final equity (last):', df['equity'].iloc[-1])
print('Sharpe:', sharpe_ratio(df['returns']))
print('Max Drawdown:', max_drawdown(df['equity']))


In [ ]:
# Visualisations
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['equity'], mode='lines', name='Equity'))
fig.update_layout(title='Triangular Backtest Equity', xaxis_title='t', yaxis_title='Equity')
fig.show()

if len(bt.trades) > 0:
    trades_df = pd.DataFrame(bt.trades)
    display(trades_df.tail())
else:
    print('No trades executed in this run.')


### Notes
- Replace synthetic snapshots with real normalized orderbook streams for production.
- Account for fees, latency, and true available depth when sizing trades.
